In [2]:
# Install necessary libraries
!pip install -q torchaudio
!pip install -q git+https://github.com/snakers4/silero-vad
!pip install -q google-colab
!pip install -q librosa soundfile pydub
!pip install -q git+https://github.com/openai/whisper.git

# To be decided
!pip install -q googletrans==4.0.0-rc1
!pip install -q gTTS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/drive/MyDrive/video-converter/example-video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# Generate my way to auto process the video

In [4]:
# Path to your video file
mp4_path = '/content/drive/MyDrive/video-converter/example-video.mp4'

# Extract audio using ffmpeg
wav_path = 'audio.wav'  # Temporary WAV file for audio

!rm -f "{wav_path}"
!ffmpeg -i "{mp4_path}" -ab 160k -ac 2 -ar 44100 -vn "{wav_path}"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:
from google.colab import drive
from IPython.display import HTML
from base64 import b64encode
import torchaudio
import torch

# Load the Silero VAD model
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

# Read the audio file
wav = read_audio(wav_path, sampling_rate=16000)

# Get speech timestamps
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=16000)

Using cache found in /root/.cache/torch/hub/snakers4_silero-vad_master


In [6]:
import whisper

# Load the Whisper model
model = whisper.load_model("base")

transcription = []
for i, timestamp in enumerate(speech_timestamps):
    start_time = timestamp['start']
    end_time = timestamp['end']

    # Extract the audio chunk
    start_sample = int(start_time )
    end_sample = int(end_time )
    chunk = wav[start_sample:end_sample]

    # Transcribe the audio chunk
    result = model.transcribe(chunk)

    # Print the transcription with timestamp
    transcription.append({
        "start": start_time,
        "end": end_time,
        "text": result['text']
    })
    print(f"[{start_time:.2f} - {end_time:.2f}] {result['text']}")


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

[3616.00 - 80352.00]  If you're a beginner in the stock market, listen up because this is one of the biggest mistakes I see beginners making today.
[81952.00 - 177632.00]  So if you're a dividend investor and you see a stock like OK, and you see it has a dividend yield of 14% you'd get hell excited, right?
[180256.00 - 346592.00]  If you don't know what that means, that's like having a savings account with a 14% interest rate. So if you were to invest $1,000 in the stock, you could expect to get paid $140 each year just for owning the stock.
[350240.00 - 376288.00]  And here are some problems I see with that.
[379936.00 - 429024.00]  First and foremost, this company is in an industry that's not doing so hot right now.
[433696.00 - 475616.00]  They have 444 million shares outstanding.
[477728.00 - 810976.00]  that to pay each of those shares $3.74 every year, which gives a company a payout ratio of a whopping 187%. A healthy payout ratio is anything under 50%, but 190% means that you're

In [7]:
from base64 import b64encode
from googletrans import Translator

translator = Translator()

for timestamp in transcription:
    start_time = timestamp['start']
    end_time = timestamp['end']

    # Translate the text to Vietnamese
    translated_text = translator.translate(timestamp["text"], dest="vi").text

    # Append to transcription with Vietnamese translation
    timestamp["vietnamese_text"] = translated_text
    print(f"[{start_time:.2f} - {end_time:.2f}] From: {timestamp['text']}\n -> To: {translated_text}")


[3616.00 - 80352.00] From:  If you're a beginner in the stock market, listen up because this is one of the biggest mistakes I see beginners making today.
 -> To: Nếu bạn là người mới bắt đầu trong thị trường chứng khoán, hãy lắng nghe vì đây là một trong những sai lầm lớn nhất mà tôi thấy người mới bắt đầu mắc phải ngày hôm nay.
[81952.00 - 177632.00] From:  So if you're a dividend investor and you see a stock like OK, and you see it has a dividend yield of 14% you'd get hell excited, right?
 -> To: Vì vậy, nếu bạn là một nhà đầu tư cổ tức và bạn thấy một cổ phiếu như OK, và bạn thấy nó có tỷ lệ cổ tức là 14% bạn sẽ rất phấn khích, phải không?
[180256.00 - 346592.00] From:  If you don't know what that means, that's like having a savings account with a 14% interest rate. So if you were to invest $1,000 in the stock, you could expect to get paid $140 each year just for owning the stock.
 -> To: Nếu bạn không biết điều đó có nghĩa là gì, điều đó giống như có một tài khoản tiết kiệm với lã

In [8]:
%mkdir -p trans_transcript

from gtts import gTTS
import os

for timestamp in transcription:
    start_time = timestamp['start']
    end_time = timestamp['end']
    translated_text = timestamp['vietnamese_text']

    # Create a filename based on the start time
    filename = f"trans_transcript/output_{start_time:.2f}.mp3"

    # Create the speech object
    tts = gTTS(text=translated_text, lang='vi')

    # Save the speech to a file
    tts.save(filename)
    timestamp["file"] = filename
    print(f"Saved Vietnamese audio for segment [{start_time:.2f} - {end_time:.2f}] to {filename}")

Saved Vietnamese audio for segment [3616.00 - 80352.00] to trans_transcript/output_3616.00.mp3
Saved Vietnamese audio for segment [81952.00 - 177632.00] to trans_transcript/output_81952.00.mp3
Saved Vietnamese audio for segment [180256.00 - 346592.00] to trans_transcript/output_180256.00.mp3
Saved Vietnamese audio for segment [350240.00 - 376288.00] to trans_transcript/output_350240.00.mp3
Saved Vietnamese audio for segment [379936.00 - 429024.00] to trans_transcript/output_379936.00.mp3
Saved Vietnamese audio for segment [433696.00 - 475616.00] to trans_transcript/output_433696.00.mp3
Saved Vietnamese audio for segment [477728.00 - 810976.00] to trans_transcript/output_477728.00.mp3
Saved Vietnamese audio for segment [814624.00 - 887776.00] to trans_transcript/output_814624.00.mp3
Saved Vietnamese audio for segment [889376.00 - 936416.00] to trans_transcript/output_889376.00.mp3


In [9]:
from IPython.display import Audio

for timestamp in transcription:
  print(f"[{timestamp['start']:.2f} - {timestamp['end']:.2f}] {timestamp['vietnamese_text']}")
  try:
    display(Audio(timestamp["file"]))
  except Exception as e:
    print(f"Error displaying audio for segment {timestamp['start']:.2f}: {e}")

  # Only want to check the first one
  break

[3616.00 - 80352.00] Nếu bạn là người mới bắt đầu trong thị trường chứng khoán, hãy lắng nghe vì đây là một trong những sai lầm lớn nhất mà tôi thấy người mới bắt đầu mắc phải ngày hôm nay.
